#SCRAPER

In [ ]:
!pip install google-play-scraper

In [ ]:
from google_play_scraper import app

import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#scrape jumlah ulasan yang diinginkan
for i in range(100):
  from google_play_scraper import Sort, reviews

  result, continuation_token = reviews(
    'com.tiktokshop.seller',
    lang='id',
    country='id',
    #sort=Sort.NEWEST,
    #sort=Sort.MOST_RELEVANT,
    count=10000,
    filter_score_with=4
  )

  df_busu = pd.DataFrame(np.array(result),columns=['review'])
  df_busu = df_busu.join(pd.DataFrame(df_busu.pop('review').tolist()))
  print(len(df_busu.index))
  if len(df_busu.index) > 1000:
    break


398
199
796
597
398
199
398
398
199
199
199
995
597
199
796
597
199
597
597
398
199
199
597
1194


In [ ]:
df_busu[['userName', 'score','at', 'content']].head()
new_df = df_busu[['userName', 'score','at', 'content']]
sorted_df = new_df.sort_values(by='at', ascending=False)
len(df_busu.index)

1194

In [ ]:
new_df.head()

,userName,score,at,content
0,Muzakir ARS,5,2024-05-24 11:42:31,sangat membantu.
1,Mas Faiq,5,2024-05-24 09:28:16,Mas Faiq aplikasi bagus pendukung UKM... terim...
2,Joko Pramono,5,2024-05-24 07:48:29,mantap jiwa
3,Lissa Dijah,5,2024-05-24 07:24:18,baik
4,Nam Nama,5,2024-05-24 07:22:13,bagus


In [ ]:
new_df.to_csv("scrapped_data2.csv", index = False)

#Cek Dublikat

In [ ]:
df_cekdublikat = pd.read_csv('tiktoksellerfix.csv')

In [ ]:
# Mengecek duplikat
duplikat = df_cekdublikat.duplicated()

# Menghitung jumlah baris yang duplikat
jumlah_duplikat = duplikat.sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")


Jumlah baris duplikat: 0


#Labeling

In [ ]:
!pip install transformers

In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=0d30c3d8e28c67d78e00c90fe4c1a719ecc3a1543a7042db3d9ebced69ea2355
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
import numpy as np
import re

In [ ]:
dflabel = pd.read_csv('tiktoksellerfix.csv')


In [ ]:
dflabel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5812 entries, 0 to 5811
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  5812 non-null   object
 1   score     5812 non-null   int64 
 2   at        5812 non-null   object
 3   content   5812 non-null   object
dtypes: int64(1), object(3)
memory usage: 181.8+ KB


In [ ]:
from googletrans.client import Translator
translator = Translator()

from transformers import pipeline
sentiment_classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
def clean(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]*)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())


In [ ]:
# Membaca kamus alay dan kamus singkatan
kamus_alay1 = pd.read_csv('kamus_alay.csv')
kamus_singkatan1 = pd.read_csv('kamus_singkatan.csv')

# Menggabungkan kamus alay dan kamus singkatan menjadi satu kamus
kamus11 = kamus_alay1.set_index('slang')['formal'].to_dict()
kamus22 = kamus_singkatan1.set_index('singkatan')['asli'].to_dict()

# Menggabungkan kedua kamus menjadi satu dictionary
kamus = {**kamus11, **kamus22}

# Fungsi untuk melakukan normalisasi teks berdasarkan kamus
def normalize_text(text, kamus):
    words = text.split()
    normalized_words = [kamus.get(word.lower(), word) for word in words]  # Menggunakan lower() untuk case insensitivity
    return ' '.join(normalized_words)

# Contoh dataframe dflabel
# dflabel = pd.DataFrame({'content': ['slang1 content example', 'singkatan1 other example']})  # Hapus atau sesuaikan dengan data asli

# Menerapkan normalisasi pada kolom content
dflabel['content'] = dflabel['content'].apply(lambda x: normalize_text(x, kamus))

# Menampilkan hasil
dflabel.head(111)


,userName,score,at,content
0,Adie,1,2024-05-24 10:04:45,"Paket banyak yang retur, kurir J&T sangat paya..."
1,Siti Aisyah,1,2024-05-24 09:51:12,1. tolong untuk keuangan bisa di transfer di r...
2,Asti Haryati,1,2024-05-24 01:37:27,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...
3,pundi artha melimpah,1,2024-05-24 00:31:06,sangat susah dan ribet
4,Agus Setiawan,1,2024-05-23 12:45:18,Lama lama capai bikin dekripsi sudah panjang u...
...,...,...,...,...
106,Fadli Abdullah,1,2024-05-05 21:22:56,Bukannya benerin jasa kirin jnt yang returanya...
107,KHAIRA CHANNEL,1,2024-05-05 18:13:24,Tidak dapat install aplikasi
108,Ardian Soraya,1,2024-05-05 14:40:04,sedikit sedikit kena pelanggaran. bagaimana ma...
109,Sudar Man,1,2024-05-05 13:53:05,bodoh


In [ ]:
dflabel['text_en'] = dflabel['content'].apply(lambda x: translator.translate(x, src='id', dest='en').text)
dflabel['clean_text'] = dflabel['text_en'].apply(clean)
dflabel['sentiment'] = dflabel['clean_text'].apply(lambda x: sentiment_classifier(x)[0])

In [ ]:
dflabel.head()

,userName,score,at,content,text_en,clean_text,sentiment
0,Adie,1,2024-05-24 10:04:45,"Paket banyak yang retur, kurir J&T sangat paya...","Many packages are returned, J&T couriers are v...",Many packages are returned J T couriers are ve...,"{'label': 'NEGATIVE', 'score': 0.9957881569862..."
1,Siti Aisyah,1,2024-05-24 09:51:12,1. tolong untuk keuangan bisa di transfer di r...,"1. Please, finances can be transferred to diff...",1 Please finances can be transferred to differ...,"{'label': 'NEGATIVE', 'score': 0.9960843324661..."
2,Asti Haryati,1,2024-05-24 01:37:27,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,I SWEAR YES OF ALL SELLER APPLICATIONS THE WOR...,I SWEAR YES OF ALL SELLER APPLICATIONS THE WOR...,"{'label': 'NEGATIVE', 'score': 0.9997407793998..."
3,pundi artha melimpah,1,2024-05-24 00:31:06,sangat susah dan ribet,very difficult and complicated,very difficult and complicated,"{'label': 'NEGATIVE', 'score': 0.9935410022735..."
4,Agus Setiawan,1,2024-05-23 12:45:18,Lama lama capai bikin dekripsi sudah panjang u...,It took a long time to make a decryption and i...,It took a long time to make a decryption and i...,"{'label': 'NEGATIVE', 'score': 0.9998130202293..."


In [ ]:
# Menggabungkan hasil analisis sentimen dengan DataFrame
dflabel1 = (
    dflabel
    .assign(
        label=lambda x: x['sentiment'].apply(lambda s: s['label']),
        score=lambda x: x['sentiment'].apply(lambda s: s['score'])
    )
    [['userName', 'content', 'text_en', 'label', 'score']]
)

In [ ]:
dflabel1.head()

,userName,content,text_en,label,score
0,Adie,"Paket banyak yang retur, kurir J&T sangat paya...","Many packages are returned, J&T couriers are v...",NEGATIVE,0.995788
1,Siti Aisyah,1. tolong untuk keuangan bisa di transfer di r...,"1. Please, finances can be transferred to diff...",NEGATIVE,0.996084
2,Asti Haryati,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,I SWEAR YES OF ALL SELLER APPLICATIONS THE WOR...,NEGATIVE,0.999741
3,pundi artha melimpah,sangat susah dan ribet,very difficult and complicated,NEGATIVE,0.993541
4,Agus Setiawan,Lama lama capai bikin dekripsi sudah panjang u...,It took a long time to make a decryption and i...,NEGATIVE,0.999813


In [ ]:
label_frequency = dflabel1['label'].value_counts()
print(label_frequency)

label
NEGATIVE    3121
POSITIVE    2691
Name: count, dtype: int64


In [ ]:
dflabel1.to_csv('tiktoksellerlabeled.csv', index=False)

#PreProcessing

In [11]:
df_pre = pd.read_csv('newlabel1.csv')
df_pre.head()

,content,sentiment
0,1. tolong untuk keuangan bisa di transfer di r...,NEGATIVE
1,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,NEGATIVE
2,sangat susah dan ribet,NEGATIVE
3,Lama lama capai bikin dekripsi sudah panjang t...,NEGATIVE
4,banyak pelanggaran kalau upload barang,NEGATIVE


In [12]:
# Mengasumsikan kolom label bernama 'label'. Ubah sesuai dengan nama kolom sebenarnya.
label_frequencies = df_pre['sentiment'].value_counts()

# Menampilkan frekuensi label
print(label_frequencies)

sentiment
NEGATIVE    1760
POSITIVE    1459
Name: count, dtype: int64


cek missing value

In [13]:
missing_values = df_pre.isnull().sum()
print(missing_values)

content      0
sentiment    0
dtype: int64


###Text Cleaning

In [14]:
import regex as re

# Fungsi untuk menghapus simbol, emotikon, dan kata "nya"
def remove_symbols_and_emoticons(text):
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus simbol
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)  # Menghapus emotikon
    text = re.sub(r'\d+', '', text)  # Menghapus angka
    text = re.sub(r'\bnya\b', '', text, flags=re.IGNORECASE)  # Menghapus kata "nya"
    return text

df_pre['text_clean'] = df_pre['content'].apply(remove_symbols_and_emoticons)
df_pre.head()


,content,sentiment,text_clean
0,1. tolong untuk keuangan bisa di transfer di r...,NEGATIVE,tolong untuk keuangan bisa di transfer di rek...
1,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,NEGATIVE,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...
2,sangat susah dan ribet,NEGATIVE,sangat susah dan ribet
3,Lama lama capai bikin dekripsi sudah panjang t...,NEGATIVE,Lama lama capai bikin dekripsi sudah panjang t...
4,banyak pelanggaran kalau upload barang,NEGATIVE,banyak pelanggaran kalau upload barang


###Case Folding

In [15]:
def lowercase(text):
    return text.lower()

df_pre['text_clean'] = df_pre['text_clean'].apply(lowercase)
df_pre['sentiment'] = df_pre['sentiment'].apply(lowercase)
df_pre.head()

,content,sentiment,text_clean
0,1. tolong untuk keuangan bisa di transfer di r...,negative,tolong untuk keuangan bisa di transfer di rek...
1,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,negative,sumpah ya dari semua aplikasi seller paling bu...
2,sangat susah dan ribet,negative,sangat susah dan ribet
3,Lama lama capai bikin dekripsi sudah panjang t...,negative,lama lama capai bikin dekripsi sudah panjang t...
4,banyak pelanggaran kalau upload barang,negative,banyak pelanggaran kalau upload barang


###Normalisasi text

In [16]:
import pandas as pd

# Membaca kamus alay dan kamus singkatan
kamus_alay = pd.read_csv('kamus_alay.csv')
kamus_singkatan = pd.read_csv('kamus_singkatan.csv')

# Menggabungkan kamus alay dan kamus singkatan menjadi satu kamus
kamus1 = kamus_alay.set_index('slang')['formal'].to_dict()
kamus2 = kamus_singkatan.set_index('singkatan')['asli'].to_dict()

# Menggabungkan dua kamus menjadi satu kamus utama
kamus_utama = {**kamus1, **kamus2}

# Fungsi untuk normalisasi kata singkatan
def normalize_abbreviations(text):
    words = text.split()
    normalized_words = [kamus_utama[word.lower()] if word.lower() in kamus_utama else word for word in words]
    return ' '.join(normalized_words)

# Menerapkan fungsi normalisasi ke kolom teks
df_pre['text_norm'] = df_pre['text_clean'].apply(normalize_abbreviations)

# Menampilkan 50 baris pertama setelah normalisasi
df_pre.head()


,content,sentiment,text_clean,text_norm
0,1. tolong untuk keuangan bisa di transfer di r...,negative,tolong untuk keuangan bisa di transfer di rek...,tolong untuk keuangan bisa di transfer di reke...
1,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,negative,sumpah ya dari semua aplikasi seller paling bu...,sumpah ya dari semua aplikasi seller paling bu...
2,sangat susah dan ribet,negative,sangat susah dan ribet,sangat susah dan ribet
3,Lama lama capai bikin dekripsi sudah panjang t...,negative,lama lama capai bikin dekripsi sudah panjang t...,lama lama capai bikin dekripsi sudah panjang t...
4,banyak pelanggaran kalau upload barang,negative,banyak pelanggaran kalau upload barang,banyak pelanggaran kalau upload barang


In [17]:
df_pre.iloc[173]

content       loginnya mudah
sentiment           positive
text_clean    loginnya mudah
text_norm     loginnya mudah
Name: 173, dtype: object

###Stopword

In [18]:
import nltk.corpus
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('indonesian')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
df_pre['text_StopWord'] = df_pre['text_norm'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Menampilkan kata stopword
removed_stopwords = set([word for word in df_pre['text_norm'].str.split().sum() if word in stop])
print("Stopwords Removed:", removed_stopwords)

df_pre.head()

Stopwords Removed: {'tengah', 'apaan', 'kalian', 'jawab', 'awal', 'tambah', 'lalu', 'tentunya', 'diinginkan', 'adanya', 'begitupun', 'sering', 'tersebut', 'menjelaskan', 'bawah', 'digunakan', 'caranya', 'siapa', 'sekaligus', 'cuma', 'mendapatkan', 'khususnya', 'hampir', 'sebelumnya', 'apakah', 'kira', 'jauh', 'pantas', 'memerlukan', 'sepertinya', 'mirip', 'macam', 'umum', 'diri', 'pasti', 'meskipun', 'segala', 'lah', 'hanya', 'sekalian', 'dikatakan', 'menanti', 'belakangan', 'telah', 'diminta', 'disebutkan', 'menurut', 'sementara', 'kemudian', 'seenaknya', 'lima', 'guna', 'padahal', 'perlu', 'selaku', 'seperti', 'adalah', 'waktu', 'sempat', 'bekerja', 'tahun', 'balik', 'misal', 'mengenai', 'depan', 'ibu', 'kecil', 'melakukan', 'tahu', 'yang', 'malahan', 'menjadi', 'semakin', 'jika', 'seseorang', 'menyiapkan', 'mempunyai', 'sehingga', 'nanti', 'dia', 'cara', 'masa', 'panjang', 'benar', 'dahulu', 'sebegitu', 'atas', 'dari', 'mereka', 'terjadi', 'tiba', 'mengira', 'diberikan', 'secara', '

,content,sentiment,text_clean,text_norm,text_StopWord
0,1. tolong untuk keuangan bisa di transfer di r...,negative,tolong untuk keuangan bisa di transfer di rek...,tolong untuk keuangan bisa di transfer di reke...,tolong keuangan transfer rekening berbeda reke...
1,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,negative,sumpah ya dari semua aplikasi seller paling bu...,sumpah ya dari semua aplikasi seller paling bu...,sumpah ya aplikasi seller buruk tuh duitnya la...
2,sangat susah dan ribet,negative,sangat susah dan ribet,sangat susah dan ribet,susah ribet
3,Lama lama capai bikin dekripsi sudah panjang t...,negative,lama lama capai bikin dekripsi sudah panjang t...,lama lama capai bikin dekripsi sudah panjang t...,capai bikin dekripsi tolak kacau
4,banyak pelanggaran kalau upload barang,negative,banyak pelanggaran kalau upload barang,banyak pelanggaran kalau upload barang,pelanggaran upload barang


###Tokenizing

In [20]:
import nltk
nltk.download('all')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:
df_pre['text_tokens'] = df_pre['text_StopWord'].apply(lambda x: word_tokenize(x))
df_pre.head()

,content,sentiment,text_clean,text_norm,text_StopWord,text_tokens
0,1. tolong untuk keuangan bisa di transfer di r...,negative,tolong untuk keuangan bisa di transfer di rek...,tolong untuk keuangan bisa di transfer di reke...,tolong keuangan transfer rekening berbeda reke...,"[tolong, keuangan, transfer, rekening, berbeda..."
1,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,negative,sumpah ya dari semua aplikasi seller paling bu...,sumpah ya dari semua aplikasi seller paling bu...,sumpah ya aplikasi seller buruk tuh duitnya la...,"[sumpah, ya, aplikasi, seller, buruk, tuh, dui..."
2,sangat susah dan ribet,negative,sangat susah dan ribet,sangat susah dan ribet,susah ribet,"[susah, ribet]"
3,Lama lama capai bikin dekripsi sudah panjang t...,negative,lama lama capai bikin dekripsi sudah panjang t...,lama lama capai bikin dekripsi sudah panjang t...,capai bikin dekripsi tolak kacau,"[capai, bikin, dekripsi, tolak, kacau]"
4,banyak pelanggaran kalau upload barang,negative,banyak pelanggaran kalau upload barang,banyak pelanggaran kalau upload barang,pelanggaran upload barang,"[pelanggaran, upload, barang]"


###Stemming

In [22]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 6.2 MB/s eta 0:00:00


In [23]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [24]:
#-----------------STEMMING -----------------
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
#import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}
hitung=0

for document in df_pre['text_tokens']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    hitung+=1
    print(hitung,":",term,":" ,term_dict[term])

print(term_dict)
print("------------------------")

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]


#script ini bisa dipisah dari eksekusinya setelah pembacaaan term selesai
df_pre['text_steamindo'] = df_pre['text_tokens'].apply(lambda x:' '.join(get_stemmed_term(x)))

4712
------------------------
1 : tolong : tolong
2 : keuangan : uang
3 : transfer : transfer
4 : rekening : rekening
5 : berbeda : beda
6 : daftarkan : daftar
7 : buka : buka
8 : toko : toko
9 : orang : orang
10 : ter : ter
11 : isi : isi
12 : pendaftaran : daftar
13 : batasi : batas
14 : akun : akun
15 : hilang : hilang
16 : lupa : lupa
17 : sandi : sandi
18 : kasih : kasih
19 : minim : minim
20 : kali : kali
21 : sumpah : sumpah
22 : ya : ya
23 : aplikasi : aplikasi
24 : seller : seller
25 : buruk : buruk
26 : tuh : tuh
27 : duitnya : duit
28 : langsung : langsung
29 : masuk : masuk
30 : saldo : saldo
31 : nunggu : nunggu
32 : berhariãâ : hari
33 : butuh : butuh
34 : modal : modal
35 : belanja : belanja
36 : pakai : pakai
37 : kecewa : kecewa
38 : aduh : aduh
39 : payah : payah
40 : susah : susah
41 : ribet : ribet
42 : capai : capai
43 : bikin : bikin
44 : dekripsi : dekripsi
45 : tolak : tolak
46 : kacau : kacau
47 : pelanggaran : langgar
48 : upload : upload
49 : barang : barang


In [25]:
df_pre.head()

,content,sentiment,text_clean,text_norm,text_StopWord,text_tokens,text_steamindo
0,1. tolong untuk keuangan bisa di transfer di r...,negative,tolong untuk keuangan bisa di transfer di rek...,tolong untuk keuangan bisa di transfer di reke...,tolong keuangan transfer rekening berbeda reke...,"[tolong, keuangan, transfer, rekening, berbeda...",tolong uang transfer rekening beda rekening da...
1,SUMPAH YA DARI SEMUA aplikasi SELLER PALING BU...,negative,sumpah ya dari semua aplikasi seller paling bu...,sumpah ya dari semua aplikasi seller paling bu...,sumpah ya aplikasi seller buruk tuh duitnya la...,"[sumpah, ya, aplikasi, seller, buruk, tuh, dui...",sumpah ya aplikasi seller buruk tuh duit langs...
2,sangat susah dan ribet,negative,sangat susah dan ribet,sangat susah dan ribet,susah ribet,"[susah, ribet]",susah ribet
3,Lama lama capai bikin dekripsi sudah panjang t...,negative,lama lama capai bikin dekripsi sudah panjang t...,lama lama capai bikin dekripsi sudah panjang t...,capai bikin dekripsi tolak kacau,"[capai, bikin, dekripsi, tolak, kacau]",capai bikin dekripsi tolak kacau
4,banyak pelanggaran kalau upload barang,negative,banyak pelanggaran kalau upload barang,banyak pelanggaran kalau upload barang,pelanggaran upload barang,"[pelanggaran, upload, barang]",langgar upload barang


###Handling missing value

In [26]:
# Memeriksa nilai kosong dalam DataFrame
missing_values = df_pre.isnull().any()

# Menampilkan kolom yang memiliki nilai kosong
print("Kolom dengan nilai kosong:")
print(missing_values[missing_values].index)

# Menampilkan jumlah nilai kosong dalam setiap kolom
print("\nJumlah nilai kosong dalam setiap kolom:")
print(df_pre.isnull().sum())

# Memeriksa kolom yang blank (hanya berisi spasi putih atau string kosong)
blank_values = (df_pre.applymap(lambda x: x.strip() if isinstance(x, str) else x) == '').any()

# Menampilkan kolom yang memiliki nilai blank
print("\nKolom dengan nilai blank:")
print(blank_values[blank_values].index)

# Menampilkan jumlah nilai blank dalam setiap kolom
print("\nJumlah nilai blank dalam setiap kolom:")
print((df_pre.applymap(lambda x: x.strip() if isinstance(x, str) else x) == '').sum())


Kolom dengan nilai kosong:
Index([], dtype='object')

Jumlah nilai kosong dalam setiap kolom:
content           0
sentiment         0
text_clean        0
text_norm         0
text_StopWord     0
text_tokens       0
text_steamindo    0
dtype: int64

Kolom dengan nilai blank:
Index(['text_StopWord', 'text_steamindo'], dtype='object')

Jumlah nilai blank dalam setiap kolom:
content            0
sentiment          0
text_clean         0
text_norm          0
text_StopWord     74
text_tokens        0
text_steamindo    77
dtype: int64


In [27]:
# Menghapus baris yang blank
df_pre_cleaned = df_pre.dropna()

# Menghapus baris yang berisi string kosong atau hanya spasi putih
df_pre_cleaned = df_pre_cleaned[~(df_pre_cleaned.applymap(lambda x: x.strip() if isinstance(x, str) else x) == '').any(axis=1)]

In [28]:
# Menghapus baris yang kosong (NaN)
df_pre_cleaned = df_pre.dropna()

# Menghapus baris yang berisi string kosong atau hanya spasi putih
df_pre_cleaned = df_pre_cleaned[~(df_pre_cleaned.applymap(lambda x: x.strip() if isinstance(x, str) else x) == '').any(axis=1)]

# Memeriksa kembali nilai kosong dalam DataFrame yang telah dibersihkan
missing_values_after_cleaning = df_pre_cleaned.isnull().any()

# Menampilkan kolom yang masih memiliki nilai kosong setelah pembersihan
print("Kolom dengan nilai kosong setelah pembersihan:")
print(missing_values_after_cleaning[missing_values_after_cleaning].index)

# Menampilkan jumlah nilai kosong dalam setiap kolom setelah pembersihan
print("\nJumlah nilai kosong dalam setiap kolom setelah pembersihan:")
print(df_pre_cleaned.isnull().sum())

# Memeriksa kolom yang blank (hanya berisi spasi putih atau string kosong) setelah pembersihan
blank_values_after_cleaning = (df_pre_cleaned.applymap(lambda x: x.strip() if isinstance(x, str) else x) == '').any()

# Menampilkan kolom yang masih memiliki nilai blank setelah pembersihan
print("\nKolom dengan nilai blank setelah pembersihan:")
print(blank_values_after_cleaning[blank_values_after_cleaning].index)

# Menampilkan jumlah nilai blank dalam setiap kolom setelah pembersihan
print("\nJumlah nilai blank dalam setiap kolom setelah pembersihan:")
print((df_pre_cleaned.applymap(lambda x: x.strip() if isinstance(x, str) else x) == '').sum())

Kolom dengan nilai kosong setelah pembersihan:
Index([], dtype='object')

Jumlah nilai kosong dalam setiap kolom setelah pembersihan:
content           0
sentiment         0
text_clean        0
text_norm         0
text_StopWord     0
text_tokens       0
text_steamindo    0
dtype: int64

Kolom dengan nilai blank setelah pembersihan:
Index([], dtype='object')

Jumlah nilai blank dalam setiap kolom setelah pembersihan:
content           0
sentiment         0
text_clean        0
text_norm         0
text_StopWord     0
text_tokens       0
text_steamindo    0
dtype: int64


In [29]:
filtered_df = df_pre_cleaned[['text_steamindo', 'sentiment']]
filtered_df.rename(columns={'text_steamindo': 'content'}, inplace=True)
filtered_df.rename(columns={'sentiment': 'sentiment'}, inplace=True)
filtered_df.head()

<ipython-input-29-8571d8336a4b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'text_steamindo': 'content'}, inplace=True)
<ipython-input-29-8571d8336a4b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'sentiment': 'sentiment'}, inplace=True)


,content,sentiment
0,tolong uang transfer rekening beda rekening da...,negative
1,sumpah ya aplikasi seller buruk tuh duit langs...,negative
2,susah ribet,negative
3,capai bikin dekripsi tolak kacau,negative
4,langgar upload barang,negative


In [30]:
filtered_df.to_csv('tiktokpre1.csv', index=False)

#Convert Xlsx to CSV coma

In [1]:
import pandas as pd

def excel_to_csv(excel_file_path, sheet_name, csv_file_path):
    # Membaca file Excel
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name, engine='openpyxl')

    # Menyimpan dataframe ke CSV dengan delimiter koma
    df.to_csv(csv_file_path, index=False, sep=',')

# Ganti 'example.xlsx' dengan path file Excel Anda
# Ganti 'Sheet1' dengan nama sheet yang ingin dikonversi
# Ganti 'output.csv' dengan path dan nama file output CSV yang diinginkan
excel_file_path = 'Book1.xlsx'
sheet_name = 'Sheet1'
csv_file_path = 'newlabel1.csv'

# Panggil fungsi untuk mengkonversi file
excel_to_csv(excel_file_path, sheet_name, csv_file_path)

print(f"File Excel '{excel_file_path}' berhasil dikonversi menjadi CSV '{csv_file_path}'")

File Excel 'Book1.xlsx' berhasil dikonversi menjadi CSV 'newlabel1.csv'


#EDA

In [31]:
df_eda = pd.read_csv('tiktokpre1.csv')
df_eda.head()

,content,sentiment
0,tolong uang transfer rekening beda rekening da...,negative
1,sumpah ya aplikasi seller buruk tuh duit langs...,negative
2,susah ribet,negative
3,capai bikin dekripsi tolak kacau,negative
4,langgar upload barang,negative


In [32]:
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   content    3142 non-null   object
 1   sentiment  3142 non-null   object
dtypes: object(2)
memory usage: 49.2+ KB


In [33]:
import plotly.express as px

# Menghitung frekuensi dari variabel sentiment
sentiment_frequencies = df_eda['sentiment'].value_counts().reset_index()
sentiment_frequencies.columns = ['sentiment', 'count']

# Membuat plot bar menggunakan Plotly
fig = px.bar(sentiment_frequencies, x='sentiment', y='count', title='Sentiment Frequencies')

# Menampilkan plot
fig.show()

###Visualisasi Positive

In [34]:
from wordcloud import WordCloud

# Memfilter data untuk hanya mengambil baris dengan nilai sentiment positif
positive_sentiment_df = df_eda[df_eda['sentiment'] == 'positive']

# Menggabungkan semua teks dari baris yang terpilih menjadi satu string
all_positive_text = ' '.join(positive_sentiment_df['content'].astype(str))

# Membuat word cloud dari teks tersebut
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_positive_text)

# Mengubah word cloud menjadi gambar yang dapat ditampilkan dengan Plotly
wordcloud_image = wordcloud.to_array()

# Membuat plot menggunakan Plotly
fig = px.imshow(wordcloud_image, title='Word Cloud of Positive Sentiment')

# Menampilkan plot
fig.show()

In [35]:
from collections import Counter
import re

# Membersihkan teks dan menghitung frekuensi kata
words = re.findall(r'\w+', all_positive_text.lower())
word_counts = Counter(words)

# Mengubah hasil ke dalam DataFrame
word_freq_df = pd.DataFrame(word_counts.items(), columns=['word', 'count']).sort_values(by='count', ascending=False)

# Membatasi hanya ke beberapa kata paling sering muncul (misalnya 20 teratas)
top_words = word_freq_df.head(23)

# Membuat plot bar menggunakan Plotly
fig = px.bar(top_words, x='word', y='count', title='Top 20 Most Frequent Words in Positive Sentiment')

# Menampilkan plot
fig.show()

###Visualisasi Negative

In [36]:
# Memfilter data untuk hanya mengambil baris dengan nilai sentiment positif
negative_sentiment_df = df_eda[df_eda['sentiment'] == 'negative']

# Menggabungkan semua teks dari baris yang terpilih menjadi satu string
all_negative_text = ' '.join(negative_sentiment_df['content'].astype(str))

# Membuat word cloud dari teks tersebut
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_negative_text)

# Mengubah word cloud menjadi gambar yang dapat ditampilkan dengan Plotly
wordcloud_image = wordcloud.to_array()

# Membuat plot menggunakan Plotly
fig = px.imshow(wordcloud_image, title='Word Cloud of negative Sentiment')

# Menampilkan plot
fig.show()

In [37]:
# Menghitung frekuensi kata
words = re.findall(r'\w+', all_negative_text.lower())
word_counts = Counter(words)

# Mengubah hasil ke dalam DataFrame
word_freq_df = pd.DataFrame(word_counts.items(), columns=['word', 'count']).sort_values(by='count', ascending=False)

# Membatasi hanya ke beberapa kata paling sering muncul (misalnya 20 teratas)
top_words = word_freq_df.head(50)

# Membuat plot bar menggunakan Plotly untuk menampilkan frekuensi kata
fig_freq = px.bar(top_words, x='word', y='count', title='Top 20 Most Frequent Words in Negative Sentiment')
fig_freq.show()